In [1]:
import sys
sys.path.append(r'D:\Personal_Python_Projects\venv\venv')

In [2]:
import pandas as pd
import os

In [11]:
import pandas as pd

def process_csv(file_path):
    df = pd.read_csv(file_path)
    
    print(f'Processing {file_path}: {df.shape}')
    
    unique_seasons = df['Season'].nunique() - 1
    
    if len(df) > 1:
        df = df.tail(1)
    else:
        print(f"DataFrame has less than 2 rows in {file_path}. Skipping processing.")
        return
    
    df.at[df.index[0], 'YEARS'] = unique_seasons
    df.at[df.index[0], 'AGE'] = None #Went back to manually edit this one

    if 'School' in df.columns:
        df['NAT_CH'] = df['School'].str.contains('UConn', case=False)
    else:
        df['NAT_CH'] = False
    
    unnamed_columns = df.columns[df.columns.str.contains('^Unnamed')]
    df = df.drop(columns=unnamed_columns)
    
    columns_to_drop = ['2P', '2PA', '2P%', 'ORB', 'DRB', 'School', 'Class', 'Conf', 'Career', 'Tm', 'Team', 'Season', 'League']
    df.drop(columns=[col for col in columns_to_drop if col in df.columns], inplace=True)
    
    for col in ['GS', 'SOS']:
        if col not in df.columns:
            df[col] = pd.NA

    df['AST'] = pd.to_numeric(df['AST'], errors='coerce')
    df['TOV'] = pd.to_numeric(df['TOV'], errors='coerce')
    df['AST/TO'] = (df['AST'] / df['TOV']).round(2)
    
    df.rename(columns={'TRB': 'RB', 'MP': 'MPG'}, inplace=True)
    
    columns_order = ['AGE', 'YEARS', 'G', 'GS', 'MPG', 'PTS', 'AST', 'RB', 'BLK', 'STL', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', 'FT', 'FTA', 'FT%', 'TOV', 'AST/TO', 'PF', 'SOS', 'NAT_CH']
    df = df[[col for col in columns_order if col in df.columns]]
    
    initial_rows = len(df)
    if len(df) > 1:
        df.drop(df.index[:-1], inplace=True)
        rows_dropped = initial_rows - len(df)
        df['YEARS'] = rows_dropped
        
    df.to_csv(file_path, index=False)
    
    print(f'Processed data saved back to {file_path}')


In [12]:
csv_dir = r'D:\Personal_Python_Projects\venv\Projects\NBA_ROTY_Prediction\Rookies'

In [13]:
for file_name in os.listdir(csv_dir):
    if file_name.endswith('.csv'):
        file_path = os.path.join(csv_dir, file_name)
        try:
            process_csv(file_path)
        except Exception as e:
            print(f'Error processing {file_path}: {e}')

Processing D:\Personal_Python_Projects\venv\Projects\NBA_ROTY_Prediction\Rookies\41.csv: (3, 30)
Processed data saved back to D:\Personal_Python_Projects\venv\Projects\NBA_ROTY_Prediction\Rookies\41.csv
Processing D:\Personal_Python_Projects\venv\Projects\NBA_ROTY_Prediction\Rookies\38.csv: (4, 30)
Processed data saved back to D:\Personal_Python_Projects\venv\Projects\NBA_ROTY_Prediction\Rookies\38.csv
Processing D:\Personal_Python_Projects\venv\Projects\NBA_ROTY_Prediction\Rookies\23.csv: (2, 27)
Processed data saved back to D:\Personal_Python_Projects\venv\Projects\NBA_ROTY_Prediction\Rookies\23.csv
Processing D:\Personal_Python_Projects\venv\Projects\NBA_ROTY_Prediction\Rookies\2.csv: (2, 27)
Processed data saved back to D:\Personal_Python_Projects\venv\Projects\NBA_ROTY_Prediction\Rookies\2.csv
Processing D:\Personal_Python_Projects\venv\Projects\NBA_ROTY_Prediction\Rookies\47.csv: (6, 30)
Processed data saved back to D:\Personal_Python_Projects\venv\Projects\NBA_ROTY_Prediction\Ro

Processed data saved back to D:\Personal_Python_Projects\venv\Projects\NBA_ROTY_Prediction\Rookies\52.csv
Processing D:\Personal_Python_Projects\venv\Projects\NBA_ROTY_Prediction\Rookies\3.csv: (2, 30)
Processed data saved back to D:\Personal_Python_Projects\venv\Projects\NBA_ROTY_Prediction\Rookies\3.csv
Processing D:\Personal_Python_Projects\venv\Projects\NBA_ROTY_Prediction\Rookies\8.csv: (2, 30)
Processed data saved back to D:\Personal_Python_Projects\venv\Projects\NBA_ROTY_Prediction\Rookies\8.csv
Processing D:\Personal_Python_Projects\venv\Projects\NBA_ROTY_Prediction\Rookies\5.csv: (2, 26)
Processed data saved back to D:\Personal_Python_Projects\venv\Projects\NBA_ROTY_Prediction\Rookies\5.csv
Processing D:\Personal_Python_Projects\venv\Projects\NBA_ROTY_Prediction\Rookies\28.csv: (3, 30)
Processed data saved back to D:\Personal_Python_Projects\venv\Projects\NBA_ROTY_Prediction\Rookies\28.csv
Processing D:\Personal_Python_Projects\venv\Projects\NBA_ROTY_Prediction\Rookies\4.csv: 

In [14]:
rookies_df = pd.read_csv(r'D:\Personal_Python_Projects\venv\Projects\NBA_ROTY_Prediction\Rookie_Names.csv', encoding='latin1')

dataframes = []

for index, row in rookies_df.iterrows():
    pick = row['Draft Pick']
    name = row['Name']
    
    stats_df = pd.read_csv(f'D:\\Personal_Python_Projects\\venv\\Projects\\NBA_ROTY_Prediction\\Rookies\\{pick}.csv', encoding='latin1')
    
    stats_df['Name'] = name
    stats_df['Draft Pick'] = pick
    dataframes.append(stats_df)

combined_df = pd.concat(dataframes, ignore_index=True)

columns_order = ['Name', 'AGE', 'Draft Pick', 'YEARS', 'G', 'GS', 'MPG', 'PTS', 'AST', 'RB', 'BLK', 'STL', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', 'FT', 'FTA', 'FT%', 'TOV', 'AST/TO', 'PF', 'SOS', 'NAT_CH']
combined_df = combined_df[[col for col in columns_order if col in combined_df.columns]]

combined_df.to_csv(r'D:\Personal_Python_Projects\venv\Projects\NBA_ROTY_Prediction\combined_rookies_stats.csv', index=False)

In [15]:
combined_df

,Name,AGE,Draft Pick,YEARS,G,GS,MPG,PTS,AST,RB,...,3PA,3P%,FT,FTA,FT%,TOV,AST/TO,PF,SOS,NAT_CH
0,Zaccharie Risacher,NaN,1,3.0,88.0,NaN,18.0,7.4,0.80,2.7,...,2.4,0.407,1.2,1.70,0.706,1.1,0.73,2.1,NaN,False
1,Alexandre Sarr,NaN,2,1.0,24.0,NaN,17.2,9.7,0.90,4.4,...,2.0,0.298,1.7,2.30,0.714,1.1,0.82,1.4,NaN,False
2,Reed Sheppard,NaN,3,1.0,33.0,5.0,28.9,12.5,4.50,4.1,...,4.4,0.521,1.6,2.00,0.831,2.0,2.25,1.9,7.46,False
3,Stephon Castle,NaN,4,1.0,34.0,30.0,27.0,11.1,2.90,4.7,...,2.2,0.267,2.4,3.20,0.755,1.5,1.93,2.4,8.7,True
4,Ron Holland,NaN,5,1.0,15.0,14.0,30.3,18.5,2.80,6.7,...,3.1,0.239,2.0,2.90,0.682,3.5,0.80,3.1,NaN,False
5,Tidjane Salaun,NaN,6,2.0,36.0,NaN,21.3,8.2,0.90,3.9,...,4.0,0.322,1.6,2.10,0.747,1.4,0.64,2.4,NaN,False
6,Donovan Clingan,NaN,7,2.0,74.0,33.0,17.6,9.8,1.00,6.4,...,0.1,0.222,1.7,3.10,0.558,0.9,1.11,1.9,8.61,True
7,Rob Dillingham,NaN,8,1.0,32.0,1.0,23.3,15.2,3.90,2.9,...,4.5,0.444,2.4,3.10,0.796,2.0,1.95,2.6,7.46,False
8,Zach Edey,NaN,9,4.0,138.0,108.0,24.9,18.2,1.30,9.6,...,0.0,0.500,4.8,6.80,0.706,1.9,0.68,2.0,9.84,False
9,Cody Williams,NaN,10,1.0,24.0,18.0,28.4,11.9,1.60,3.0,...,1.7,0.415,2.3,3.20,0.714,2.0,0.80,2.0,8.36,False


In [ ]:
#Had to manually edit AGE and NAT_CH columns